In [ ]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit import IBMQ, Aer, execute
from qiskit.providers.ibmq import least_busy
import sympy

def quantum_true_random_number_generation(num_qubits, shots):
    circuit = QuantumCircuit(num_qubits, num_qubits)
    for i in range(num_qubits):
        circuit.h(i)
    circuit.measure(range(num_qubits), range(num_qubits))
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(circuit, simulator, shots=shots)
    result = job.result()
    counts = result.get_counts(circuit)

    # Przekształcenie wyników na liczby
    random_numbers = []
    for binary, count in counts.items():
        number = int(binary, 2)
        random_numbers.append(number)

    return random_numbers

# Wywołanie procedury TRNG
num_qubits = 2048
shots = 2
random_numbers = quantum_true_random_number_generation(num_qubits, shots)


random_num1 = sympy.prevprime(random_numbers[0])
random_num2 = sympy.prevprime(random_numbers[1])
print("[", random_num1, ",", random_num2, "]")



In [ ]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding, utils

def gcd(a, b):
    if b > 0:
        return gcd(b, a % b)
    return a

def find_mod_inverse(a, m):
    if gcd(a, m) != 1:
        return None
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m

    while v3 != 0:
        q = u3 // v3
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

# https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/
def createKeys():
    import random
    p = random_num1
    q = random_num2
    
    n = p * q
    phiN = (p - 1) * (q - 1)

    while True:
        e = random.randrange(1, phiN)
        g = gcd(e, phiN)
        if g == 1:
            break

    # print(e, n)
    # d = pow(e, -1, phiN) #throws error
    
    d = find_mod_inverse(e, phiN)

    dmp1 = d % (p-1)
    dmq1 = d % (q-1)
    iqmp = pow(q, -1, p)
    publicKey = rsa.RSAPublicNumbers(e=e, n=n).public_key()
    privateKey = rsa.RSAPrivateNumbers(
        p=p,
        q=q,
        d=d,
        dmp1=dmp1,
        dmq1=dmq1,
        iqmp=iqmp,
        public_numbers=rsa.RSAPublicNumbers(e=e, n=n)
    ).private_key()

    return privateKey, publicKey
    
def RSAPrivateToPEM(privateKey):
    pemPrivateKey = privateKey.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption(),
    )
    
    return pemPrivateKey

# Convert the raw public key to PEM form
def RSAPublicToPEM(publicKey):
    pemPublicKey = publicKey.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo,
    )

    return pemPublicKey

myPrivateKey, myPublicKey = createKeys()
PEMPrivateKey = RSAPrivateToPEM(myPrivateKey)
PEMPublicKey = RSAPublicToPEM(myPublicKey)

# print(PEMPrivateKey)
# print(PEMPublicKey)

with open("private_key.pem", "wb") as pem_file:
  pem_file.write(PEMPrivateKey)
with open("public_key.pem", "wb") as pem_file:
  pem_file.write(PEMPublicKey)

# print("---SIGNATURE [hex]---\n", mySignKey.hex(), "\n---ENDOF SIGNATURE---")
# print("public key matches sign?", verifySignature(message, mySignKey, myPublicKey))